<a href="https://colab.research.google.com/github/noamkasten/QuantumProcessMaps/blob/main/5_qubit_error_correction_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sympy
import sympy as sp
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import TensorProduct
from sympy import I, simplify, collect, factor, Array, cos, sin, pi, lambdify, sqrt, Trace, exp
from sympy.physics.quantum.trace import Tr

# Define symbolic variables
q,p,t,f = sympy.symbols('q p t f', real=True)

In [ ]:
Id = sympy.Matrix(np.array([[1, 0],[0, 1]]))
X = sympy.Matrix(np.array([[0, 1],[1, 0]]))
Y = sympy.Matrix(np.array([[0, -I],[I, 0]]))
Z = sympy.Matrix(np.array([[1, 0],[0, -1]]))
H = 1/sqrt(2) * sympy.Matrix(np.array([[1, 1],[1, -1]]))
Rx = sympy.Matrix(np.array([[cos(t/2), -I*sin(t/2)], [-I*sin(t/2), cos(t/2)]]))
Rz = sympy.Matrix(np.array([[exp(-I*f/2), 0], [0, exp(I*f/2)]]))
ZERO_MAT = sympy.Matrix(np.array([[1,0],[0,0]]))
ONE_MAT = sympy.Matrix(np.array([[0,0],[0,1]]))


ALL_ZERO_QUBIT = sympy.Matrix(np.array([[0,0],[0,0]]))
ALL_ZEROS_5 = TensorProduct(ALL_ZERO_QUBIT, ALL_ZERO_QUBIT, ALL_ZERO_QUBIT, ALL_ZERO_QUBIT, ALL_ZERO_QUBIT)



def c_gate(c_idx, t_idx, controlled_gate, num_qubits=5 + 1):
  '''
  get 2 control and target indices, create a dict and return a CNOT matrix
  num_qubits = 5 + 1 to represent the 5 code and one control qubit to be measured
  '''
  zero_part_matrices = []
  one_part_matrices = []

  for idx in range(num_qubits):
    if idx == c_idx:
      zero_part_matrices.append(ZERO_MAT)
      one_part_matrices.append(ONE_MAT)
    elif idx == t_idx:
      zero_part_matrices.append(Id)
      one_part_matrices.append(controlled_gate)
    else:
      zero_part_matrices.append(Id)
      one_part_matrices.append(Id)

  zero_part = TensorProduct(*zero_part_matrices)
  one_part = TensorProduct(*one_part_matrices)
  cnot_gate =  zero_part + one_part
  return cnot_gate


def noisy_c_gate(rho_input, c_idx, t_idx, controlled_gate, num_qubits= 5 + 1):
  cgate_unitary = c_gate(c_idx=c_idx, t_idx=t_idx, controlled_gate=controlled_gate, num_qubits = num_qubits)
  return q * rho_input + (1-q) * cgate_unitary * rho_input * Dagger(cgate_unitary)

def does_commuttator_different_from_zero(A,B):
  commutation = A * B - B * A
  if commutation == ALL_ZEROS_5:
    return 0
  else:
    return 1

def measure_syndrom(density_matrix_errored, syndrom_list_by_qubit=[Id,Id,Id,Id,Id]):
  """
  gets a syndrom by qubit "syndrom_list_by_qubit" and then the density matrix with the error "density_matrix_errored"
  and then caclulate the noisy syndrom measurements probabilities.
  """
  ANCILLA_QUBIT = 5
  H_ancilla = TensorProduct(Id, Id, Id, Id, Id, H)

  ####### SYNDROME START #######
  density_matrix_errored_with_ancilla = TensorProduct(density_matrix_errored, ZERO_MAT)
  density_matrix_before_syndrome = H_ancilla * density_matrix_errored_with_ancilla * Dagger(H_ancilla)
  current_density_matrix = density_matrix_before_syndrome

  for qubit_idx in range(ANCILLA_QUBIT):
    cur_cgate = syndrom_list_by_qubit[qubit_idx]
    if cur_cgate != Id:
      current_density_matrix = noisy_c_gate(current_density_matrix, c_idx=ANCILLA_QUBIT, t_idx=qubit_idx, controlled_gate=cur_cgate, num_qubits=5+1)

  syndrom_before_meas = H_ancilla * current_density_matrix * Dagger(H_ancilla)
  ####### SYNDROME END #######

  ####### Measure START #######
  MEASURE_ANCILLA_ONE = TensorProduct(Id, Id, Id, Id, Id, ONE_MAT)
  projected_syndrome = MEASURE_ANCILLA_ONE * syndrom_before_meas * MEASURE_ANCILLA_ONE
  probability_to_measure_1 = projected_syndrome.trace()
  ####### Measure END #######

  return sympy.simplify(probability_to_measure_1)


def get_all_syndroms_probabilities_to_get_1(density_matrix_err):
  probabilities_to_get_1 = [0, 0, 0, 0]
  list_of_syndromes = [[Z, X, X, Z, Id], [X, X, Z, Id, Z], [X, Z, Id, Z, X], [Z, Id, Z, X, X]]
  for idx_probability in range(len(probabilities_to_get_1)):
    probabilities_to_get_1[idx_probability] = measure_syndrom(density_matrix_errored=density_matrix_err, syndrom_list_by_qubit=list_of_syndromes[idx_probability])
  return probabilities_to_get_1


def make_error_and_get_proability_of_syndrome(density_matrix_before_error, error_list=[Id,Id,Id,Id,Id]):
  """
  :type error_list: sympy.Matrix
  """
  stabilizer_0_list = [Z, X, X, Z, Id]
  stabilizer_1_list = [X, X, Z, Id, Z]
  stabilizer_2_list = [X, Z, Id, Z, X]
  stabilizer_3_list = [Z, Id, Z, X, X]
  stabilizers_list = [stabilizer_0_list, stabilizer_1_list, stabilizer_2_list, stabilizer_3_list]

  stabilizer_0 = TensorProduct(*stabilizer_0_list)
  stabilizer_1 = TensorProduct(*stabilizer_1_list)
  stabilizer_2 = TensorProduct(*stabilizer_2_list)
  stabilizer_3 = TensorProduct(*stabilizer_3_list)
  stabilizers = [stabilizer_0, stabilizer_1, stabilizer_2, stabilizer_3]
  # adding fixing channels when error accoured

  current_probability_channel = 1
  error_unitary = TensorProduct(*error_list)

  errored_density_matrix = error_unitary*density_matrix_before_error*Dagger(error_unitary)

  for idx_stabilizer, stabilizer in enumerate(stabilizers):
    syndrom_not_commutative = does_commuttator_different_from_zero(stabilizer, error_unitary)
    probability_to_get_1_on_current_syndrome = measure_syndrom(errored_density_matrix, syndrom_list_by_qubit=stabilizers_list[idx_stabilizer])

    if syndrom_not_commutative:
      current_probability_channel *= probability_to_get_1_on_current_syndrome
    else:
      current_probability_channel *= 1 - probability_to_get_1_on_current_syndrome

  return sympy.simplify(current_probability_channel)



###### START ENCODING 0L ######
stabilizer_0 = TensorProduct(Z, X, X, Z, Id)
stabilizer_1 = TensorProduct(X, X, Z, Id, Z)
stabilizer_2 = TensorProduct(X, Z, Id, Z, X)
stabilizer_3 = TensorProduct(Z, Id, Z, X, X)
Identity_5_qubits = TensorProduct(Id, Id, Id, Id, Id)
prepare_logic_qubit = 0.25 * (Identity_5_qubits+stabilizer_0) * (Identity_5_qubits+stabilizer_1) * (Identity_5_qubits+stabilizer_2) * (Identity_5_qubits+stabilizer_3)


ket_0 = sympy.Matrix([1, 0])


####### ROTATION START #######
rotated_state_from_zero_theta = Rx*ket_0
rotated_state = Rz*rotated_state_from_zero_theta
print("rotated_state:", rotated_state)
####### ROTATION FINISH #######


####### GHZ START #######

ket_5_qubit_after_rotation_qubit_0 = TensorProduct(rotated_state, ket_0, ket_0, ket_0, ket_0)

cnot_0_1 = c_gate(c_idx=0, t_idx=1, controlled_gate=X, num_qubits=5)
cnot_0_2 = c_gate(c_idx=0, t_idx=2, controlled_gate=X, num_qubits=5)
cnot_0_3 = c_gate(c_idx=0, t_idx=3, controlled_gate=X, num_qubits=5)
cnot_0_4 = c_gate(c_idx=0, t_idx=4, controlled_gate=X, num_qubits=5)

GHZ_processing = cnot_0_1 * cnot_0_2 * cnot_0_3 * cnot_0_4
before_encoding_5_qubit = GHZ_processing * ket_5_qubit_after_rotation_qubit_0
print(before_encoding_5_qubit)
####### GHZ FINISH #######

encoded_state = prepare_logic_qubit * before_encoding_5_qubit
###### FINISH ENCODING 0L ######


density_matrix_encoded = encoded_state * Dagger(encoded_state)

####### ERROR START #######

joint_probability = 0

joint_probability += (1-p) * make_error_and_get_proability_of_syndrome(density_matrix_before_error=density_matrix_encoded)

fixing_gates = [X, Y, Z]
# adding error accoured
for qubit_idx in range(5):
  for fixing_gate in fixing_gates:
    cur_err = [Id, Id, Id, Id, Id]
    cur_err[qubit_idx] = fixing_gate
    probability_to_recognize_successfull_the_error = make_error_and_get_proability_of_syndrome(density_matrix_before_error=density_matrix_encoded, error_list=cur_err)
    print(probability_to_recognize_successfull_the_error)
    print(probability_to_recognize_successfull_the_error.subs({q: 0}).evalf())
    joint_probability += (p/15) * probability_to_recognize_successfull_the_error

####### ERROR END #######


joint_probability = sympy.simplify(joint_probability)
print(joint_probability)


rotated_state: Matrix([[exp(-I*f/2)*cos(t/2)], [-I*exp(I*f/2)*sin(t/2)]])
Matrix([[exp(-I*f/2)*cos(t/2)], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [-I*exp(I*f/2)*sin(t/2)]])
9.0*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)**2*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**2
1.00000000000000
81.0*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**4
1.00000000000000
9.0*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)**2*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**2
1.00000000000000
(-q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) - 1)**3*(2.0*q**3 - 3.0*q**2 + 2.0*q - 1.0)
1.00000000000000
9.0*(-q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) - 1)*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**2*(2.0*q**3 - 3.0*q**2 + 2.0*q - 1.0)
1.00000000000000
9.0*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)*

# Result Evaluation

In [ ]:
equation = -4*p*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)**3*(2.0*q**3 - 3.0*q**2 + 2.0*q - 1.0)/15 + 3.6*p*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)**2*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**2 - 2.4*p*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**2*(2.0*q**3 - 3.0*q**2 + 2.0*q - 1.0) + 5.4*p*(0.666666666666667*q**3 - q**2 + 0.666666666666667*q - 0.333333333333333)**4 + (1 - p)*(q*(1.0*q**3 - 2.0*q**2 + 3.0*q - 2.0) + 1)**4
equation_expanded = sp.expand(equation)
print(sp.collect(equation_expanded, [q, p]))
